In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=e7e5f8cc3227cc9603369a52511b5ccc00e481cc4334620a90ec0f14bdd47095
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 722 (delta 0), reused 0 (delta 0), pack-reused 719
Receiving objects: 100% (722/722), 87.94 MiB | 36.47 MiB/s, done.
Resolving deltas: 100% (456/456), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
from data_utils import (PpgDaliaExtractor, FormatPPGDalia)

extractor = PpgDaliaExtractor(DATA_DIR)
ppg_dalia_formatter = FormatPPGDalia()
dfs_train = [ppg_dalia_formatter.transform(extractor.extract_subject(i)) for i in range(1,16)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [10]:
git_pull()

import importlib

import PPG
importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.TrainerXY)
importlib.reload(PPG.FullTrainer)
importlib.reload(PPG.TrainerIS)

from PPG import FullTrainer

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 10 (delta 5), reused 10 (delta 5), pack-reused 0
Unpacking objects: 100% (10/10), done.
From github.com:davipeag/HeartRateRegression
   896229a..e9f4bfe  master     -> origin/master
Updating 896229a..e9f4bfe
Fast-forward
 Ensembles.py               | 22 ++++++++++++++++
 Optimization/Optimizers.py | 34 +++++++++++++++++++++++++
 PPG/FullTrainer.py         | 62 ++++++++++++++++++++++++++++++++++++++++++++++
 test.py                    |  5 ++--
 4 files changed, 121 insertions(+), 2 deletions(-)
 create mode 100644 Ensembles.py
 create mode 100644 Optimization/Optimizers.py


In [11]:
import random 
options = {
  'nfeatures': [4],
  'conv_filters': [64, 128, 256],
  'nconv_layers': [1, 2, 3, 4],
  'conv_dropout': [0, 0.1, 0.25,],
  'nenc_layers': [1, 2, 3, 4],
  'ndec_layers': [2, 3, 4],
  'nhead': [1, 2, 4],
  'feedforward_expansion': [1, 2, 4],
  'nlin_layers': [2,3,4],
  'lin_size': [16, 32, 64],
  'lin_dropout': [0, 0.25, 0.5],
  'lr': [0.001, 0.0001],
  'weight_decay': [0, 0.00001],
  'batch_size': [128, 256],
  'ts_sub': [0],
  'val_sub': [4]
 }

def choose(options):
  choice = dict()
  for k,v in options.items():
    choice[k] = random.choice(v)
  return choice


In [ ]:
from  Optimization.Optimizers import RandomSearch
import json

filename = "joint_attentiontion_full_trainer_results.json"
save_path = os.path.join(STORE_DIR, filename)

try:
  with open(save_path, "r") as f:
    results = json.load(f)
except FileNotFoundError:
  results = list()
 
searcher = RandomSearch(FullTrainer.JointValAttentionFullTrainer(dfs_train, args["device"]), options)
searcher.results = results
while True:
  searcher.fit(1)
  with open(save_path, "w") as f:
    json.dump(searcher.results, f)




####
Failed: {'nfeatures': 4, 'conv_filters': 256, 'nconv_layers': 3, 'conv_dropout': 0.1, 'nenc_layers': 3, 'ndec_layers': 3, 'nhead': 1, 'feedforward_expansion': 4, 'nlin_layers': 2, 'lin_size': 32, 'lin_dropout': 0.5, 'lr': 0.001, 'weight_decay': 0, 'batch_size': 128, 'ts_sub': 0, 'val_sub': 4}
###
best val epoch: 1
[1/30]: loss_train: 17.158 loss_val 16.912 loss_ts 11.474


In [15]:
from PPG.FullTrainer import JointValAttentionFullTrainer
while True:
  full_trainer = FullTrainer.JointValAttentionFullTrainer(dfs_train, args["device"])
  out = full_trainer.train(**choose(options))
  print(out["args"], out["metric"])

best val epoch: 1
[1/30]: loss_train: 15.961 loss_val 15.448 loss_ts 10.991
best val epoch: 2
[2/30]: loss_train: 15.638 loss_val 15.230 loss_ts 10.613
best val epoch: 3
[3/30]: loss_train: 14.188 loss_val 13.952 loss_ts 9.869
best val epoch: 4
[4/30]: loss_train: 13.723 loss_val 13.501 loss_ts 9.882


KeyboardInterrupt: ignored

In [ ]:
full_trainer = FullTrainer.AttentionFullTrainer(dfs_train, args["device"], 0, 1)

full_trainer.train()

In [ ]:
from PPG import UtilitiesDataXY 


transformers = PPG.AttentionDefaults.get_preprocessing_transformer()
make_loaders = UtilitiesDataXY.DataLoaderFactory(transformers, dfs_train).make_loaders

loader_tr, loader_val, loader_ts = make_loaders(ts_sub=0, val_sub=1)

In [ ]:
from PPG.Models import SnippetConvolutionalTransformer

net = SnippetConvolutionalTransformer().to(args["device"])

# x,y = next(iter(loader_tr))

# p = net(x)

criterion = nn.MSELoss().to(args["device"])# nn.L1Loss().to(args["device"]) #nn.CrossEntropyLoss().to(args["device"])
optimizer = torch.optim.Adam(net.parameters(), lr=args["lr"],
                             weight_decay=args["weight_decay"])


In [ ]:
from PPG.TrainerXY import (EpochTrainerXY, MetricsComputerXY, TrainHelperXY)
from preprocessing_utils import ZTransformer2

epoch_trainer = EpochTrainerXY(net, optimizer, criterion, args["device"])
ztransformer = ZTransformer2(['heart_rate', 'wrist-ACC-0', 'wrist-ACC-1', 'wrist-ACC-2',
              'wrist-BVP-0', 'wrist-EDA-0', 'wrist-TEMP-0', 'chest-ACC-0',
              'chest-ACC-1', 'chest-ACC-2', 'chest-Resp-0'])
metrics_comuter = MetricsComputerXY(ztransformer)

train_helper = TrainHelperXY(epoch_trainer, loader_tr, loader_val, loader_ts, metrics_comuter.mae)

In [ ]:
train_helper.train(30)